In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import pandas as pd
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException, ElementNotInteractableException  #(클릭시 없을때, 엘리멘트 자체가 없을떄, 엘리멘트가 상호작용을 못할때 )
import os
import shutil 
from urllib.request import urlretrieve
import re



#웹브라우저를 띄우지 않고 진행하기 위한 설정
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

### get_politician_info(dir_name, wd) : 국회의원 정보 스크래핑

In [2]:
def get_politician_info(dir_name, wd):
    profile_info = wd.find_element(By.CSS_SELECTOR, '#contents > div.sp-person.contents > section.person-info.spacial-page.cl')
    cols = profile_info.find_elements(By.TAG_NAME, 'dt')
    columns = [col.text for col in cols]
    
    infos = profile_info.find_elements(By.TAG_NAME, 'dd')
    detail_info = [info.text for info in infos]
    
    info_dic = {col:info for col, info in zip(columns, detail_info)}
    
    return info_dic

### get_news_quote(dir_name, name, wd): 뉴스 인용문 스크래핑

In [3]:
def get_news_quote(dir_name, name, wd):
    
    page_no = 0
    news_count = 0
    news_total = wd.find_element(By.ID, 'newsInQoutTotalCount').text[1:-2]
    news_file = open(dir_name + '/' + name + '뉴스 인용문.txt','w')
    
    while True:
        try:
            page_no += 1
            print(f"---------(뉴스 인용문 {page_no} 페이지)----------")
            time.sleep(1)
            
            box_list = wd.find_element(By.ID, 'newsInQuotList')
            quotoes = box_list.find_elements(By.CLASS_NAME, 'title')
            
            time.sleep(1)
            
            for quoto in quotoes:
                news_file.write(quoto.text + '\n')
                news_count += 1
                print(f"{news_count}/{news_total}", [quoto.text])
                print()
#                 if news_count >= news_max:
#                     break


            paging = wd.find_element(By.ID, 'newsInQuotListPaging')

            next_button = paging.find_element(By.CSS_SELECTOR, '#newsInQuotListPaging > a.page-next.page-link')
            if next_button.get_attribute('onclick') == None:
                break
                    
                         
            pagination = paging.find_element(By.CSS_SELECTOR, '#newsInQuotListPaging > a.page-next.page-link').click()
            time.sleep(1)

                    
#             if news_count >= news_max:
#                 break
                

        except Exception as e:
            print(e)
            break
            
    news_file.close()

### scraping(): 스크래핑 함수

In [4]:
def scraping(dir_name, politician_max=3):
    page_no = 0
    politician_count = 0
    politician_df = pd.DataFrame()
    
    wd = webdriver.Chrome('chromedriver', options=chrome_options)
    wd.execute_script('window.open("about:blank", "_blank");')
    tabs = wd.window_handles
    
    while True:
        
        try:
            page_no += 1
            wd.switch_to.window(tabs[0])

            url = f"https://www.bigkinds.or.kr/v2/depthAnalysis/assembly.do?page={page_no}"
            wd.get(url)

            politician_total = wd.find_element(By.XPATH, '//*[@id="contents"]/section[1]/div/div/div[1]/div[2]').text
            politician_items = wd.find_elements(By.CSS_SELECTOR, '#contents > section.spacial-person.spacial-page > div > ul > li')

            if not politician_items:
                break

            for item in politician_items:
                wd.switch_to.window(tabs[0])

                politician_count += 1

                print('--------------------------------------')
                print(f"[국회의원 {politician_count}/{politician_total}명]" )
                name = item.find_element(By.CLASS_NAME, 'sp-kname').text
    #             name = name[:3] 이름 자를때 쓰자!
                print(name)
                
                detail_link = item.find_element(By.TAG_NAME, 'a').get_attribute('href')
#                 print(detail_link)
                wd.switch_to.window(tabs[1])
                wd.get(detail_link)
                
                print("[프로필 이미지 다운로드]")
                profile_image = wd.find_element(By.CLASS_NAME, 'thumb')
                image_src = profile_image.find_element(By.TAG_NAME, 'img').get_attribute('src')
                file_name = dir_name + '/' + name + '.jpg'
                print(file_name)
                urlretrieve(image_src, file_name)
                
                print("[기본 정보 스크래핑]")
                politician_dic = get_politician_info(dir_name, wd)
                print("    ",politician_dic)
                politician_df = politician_df.append(politician_dic, ignore_index=True)
                
                print("[뉴스 인용문 스크래핑]")
                tab = wd.find_element(By.CSS_SELECTOR, 'li.analysisTab-01.ui-state-active').find_element(By.TAG_NAME, 'a').click()
                time.sleep(1)
                get_news_quote(dir_name, name, wd)
                
    

                if politician_count >= politician_max:
                    break
            if politician_count >= politician_max:
                    break
        except AttributeError as e:
            print(e)
            break
            
        except NoSuchElementException as e:
            print(e)
            break
    
    wd.close()
    
    return politician_df


### 스크래핑 시작

In [5]:
dir_name = './politicial' #폴더 생성
if os.path.isdir(dir_name): #폴더가 있으면
    shutil.rmtree(dir_name) #지움

os.makedirs(dir_name)
print(f"'DIR_NAME': {dir_name} 디렉토리 생성")

politician_df = scraping(dir_name)

'DIR_NAME': ./politicial 디렉토리 생성
--------------------------------------
[국회의원 1/295명]
강기윤(姜起潤)
[프로필 이미지 다운로드]
./politicial/강기윤(姜起潤).jpg
[기본 정보 스크래핑]
     {'지역구': '경남 창원시성산구', '당선기록': '재선(19대, 21대)', '소속위원회': '보건복지위원회', '사무실전화': '02-784-1751', '보좌관': '김홍광 , 한영애', '이메일주소': 'ggotop@naver.com', '홈페이지': 'http://blog.naver.com/ggotop', '경력': '[학력] 마산공고(26회) 창원대학교 행정학과 중앙대학교 행정대학원 지방의회과 석사 창원대학교 대학원 행정학 박사 [경력] 현) 국회 보건복지위원회 국민의힘 간사 현) 국민의힘 소상공인살리기 특별위원회 부위원장 현) 국민의힘 코로나19 대책 특별위원회 위원 미래통합당 경남도당 민생특위 위원장 제19대 국회의원 (새누리당/경남 창원시 성산구) 새누리당 원내부대표'}
[뉴스 인용문 스크래핑]
---------(뉴스 인용문 1 페이지)----------
1/9 ['영역 간 다툼 없이 할 수 있는 것을 정부가 고민했어야 한다']

2/9 ['정부가 재택치료를 고려해 먹는 코로나19 치료제 확보에 치중하고 있는데, (정부가 선구매 계약 진행 중인) 머크사의 치료제는 임신부에게 위험할 수 있다는 우려가 제기됐다 해외에서는 (국내 사용 중인) 렉키로나주와 소트로비맙 등 다양한 항체치료제 물량 확보에 나서고 있는 만큼 우리 정부도 치료제를 다양하게 확보해 환자 맞춤 치료를 해야 할 것']

3/9 ['정부가 재택치료를 고려해 먹는 코로나19 치료제 확보에 치중하고 있는데, (정부가 선구매 계약 진행 중인) 머크사의 치료제는 임신부에 위험할 수 있다는 우려가 제기됐다 해외에서는 (국내 사용 중인) 렉키로나주와 소트로비맙 등의 다양한 항체치료제 물량 확보에 나서고 있는 만큼 정부도 

---------(뉴스 인용문 2 페이지)----------
11/70 ['이번 기자회견의 성격상 반드시 포함되어야 할 허위이력 기재에 대한 구체적 사실 관계가 적시되지 않았을 뿐더러 오히려 부부간 사생활 이야기가 중심을 이룬것 같다 물론 안타까운 부분도 있지만 그렇다고 해서 김씨 허위이력 기재 포함 모든 것이 끝난 것은 아니다. 김씨의 모든 의혹은 윤석열 검찰총장의 공정과 정의라는 잣대로 수사를 해야한다. 사과했다고 죄가 덮..']

12/70 ['공정과 정의를 외치고, 특수통 검사로 이름을 날렸던 윤석열 후보는 제2부속실을 없애기에 앞서 부인과 관련한 허위 이력 의혹, 그리고 양평 게이트 의혹 등에 대해서 진솔한 소명을 하는 것이 먼저']

13/70 ['안양예술공원과 서울대 관악수목원은 안양이 얼마나 자연친화적이고 문화친화적인 도시인지를 알려주는 전초기지 역할을 한다 수목원을 개방하면 지역상권이 활성화되고 안양시민뿐만 아니라 수도권 시민들이 쉴 수 있는 명소가 될 것 안양시와 서울대와 협력해 수목원 본연의 역할을 충실히 할 수 있는 여건도 마련하겠다']

14/70 ['수목원 개방을 통해 지역상권 활성화는 물론 수도권 시민들이 함께 휴식할 수 있는 장소가 될 것 안양시와 서울대가 협력해 수목원 본연의 역할을 충실히 할 수 있는 여건을 마련하겠다']

15/70 ['윤 후보 장모 최은순씨와 가족회사인 이에스아이엔디(ESI&D)가 양평 공흥지구 땅을 사들이기 이전인 2004년부터 일대 토지를 차명으로 보유한 정황이 있다 윤 후보의 장모 측이 안씨가 다른 마음을 먹고 땅을 처분하는 걸 막기 위해 가등기를 설정했을 가능성이 높다 ESI&D 명의로 매입한 8개 필지는 2006년 12월 6일 매매가 이뤄졌고, 그 후인 같은..']

16/70 ['2006년 ESI&D가 안모씨로부터 매입한 임야가 이후 최씨가 사들인 농지보다 훨씬 비싼 것은 누가봐도 정상적인 거래라 보기 어렵다 앞서 민주당 현안대응TF에서 고발한 병산리 토지들과 소유명의자의 땅에 가등기를 설정하는 구

In [6]:
politician_df

,지역구,당선기록,소속위원회,사무실전화,보좌관,이메일주소,홈페이지,경력
0,경남 창원시성산구,"재선(19대, 21대)",보건복지위원회,02-784-1751,"김홍광 , 한영애",ggotop@naver.com,http://blog.naver.com/ggotop,[학력] 마산공고(26회) 창원대학교 행정학과 중앙대학교 행정대학원 지방의회과 석사...
1,대구 동구을,초선(21대),"국방위원회 , 정치개혁 특별위원회",02-784-5275,"박홍규 , 정운태",kds21341@naver.com,,[학력] 송정초등학교 안심중학교 경북대학교 사범대학 부속고등학교 영남대학교 경영대학...
2,경기 안양시만안구,초선(21대),"교육위원회 , 예산결산특별위원회",02-784-2747~9,"서용선 , 안홍식",mainsail440@daum.net,https://blog.naver.com/dulipapa,2018. 10. ~ 2019. 5. 민주연구원 자치발전연구센터 본부장 2016. ...
